**ASSIGNMENT 1 - EMPIRICAL STUDY OF KNAPSACK PROBLEM**

**1. Group Description**

Group Number: \\
Member Names: \\
Member Student Numbers: \\

**2. Knapsack Problem**

Give a description of the problem tackled.

**3. Dataset**

Give a description of the dataset used with references.  

**Import important libraries**

In [23]:
import pandas as pd
import itertools
import numpy as np
import itertools
import time

**Read Dataset**

As outlined in the project description, it should be possible for the correctors to execute your notebook without requiring any downloads.

To facilitate access to the dataset without the need for downloads, you can upload it to a public GitHub repository and provide a link to the raw version of the dataset.

The link to the raw version is as follows:
*https://raw.githubusercontent.com/GITHUB_USERNAME/REPOSITORY_NAME/main/DATASETNAME.csv*

For example:

https://raw.githubusercontent.com/baharin/KnapsackProblem/main/knapsack_5_items.csv

Now provide the link to YOUR dataset and read the dataset using pandas:

In [24]:
url = "https://raw.githubusercontent.com/Fengshou-Xu/slssdhs-zyjsdzs/main/knapsack_5_items.csv"

dataset = pd.read_csv(url)

Let's see what are the columns of the dataset? :

In [25]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

As we expected, we have columns for weights, costs, capacity, best picks and best price for all the instances.

Now let's see the first 10 entries (rows):

In [26]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19.0
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47.0
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36.0
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19.0
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44.0
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48.0
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37.0
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13.0
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35.0
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45.0


**Preprocessing Step**

Typically, the initial step in any project that involves reading and handling data is data preprocessing and cleansing.

In our dataset, we expect the entries in the "Weights," "Prices," and "Best Picks" columns to be in the form of arrays of floats or integers, like this: [45, 40, 42, 38, 10]

However, when you read each entry using pandas, they will be in a form of a string: "[45 40 42 38 10]"

So we need to convert these strings into "arrays of floats or integers." You can utilize the function provided below for this purpose:


In [27]:
def string_to_list(string):

  string_list = string.strip('[]').split()

  float_list = [float(element) for element in string_list]

  return float_list

Furthermore, it's possible that certain rows in the dataset contain empty values in specific columns. We also aim to eliminate these rows as they do not provide any useful information. We use dropna() function to do so:

In [28]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

Now it's time to implement the search algorithms. For each algorithm, a template is provided to you. You can modify this template if you want. But first you should try to go look at all the parameters used, as they are all important. You can also define any number of auxiliary functions you want.


**4. Generate and Test**

This part we will have a exhaustive_search algorithm, and we will use it as a standard to compare other algorithm

In [29]:
def gen_and_test(data):
  weights = data['Weights']
  prices = data['Prices']
  capacity = data['Capacity']

  best_solution_price , best_solution = exhaustive_search(weights, prices, capacity)


  return best_solution_price, best_solution

def exhaustive_search(weights, values, capacity) : # I misunderstood the function of gen_and_test at first, so I wrote this function to do exhaustive search
      n = len(weights) # number of items
      best_value = 0 # value of the best combination
      best_combination = [] # indices of the best combination

    
      for r in range(n + 1):
        for subset in itertools.combinations(range(n), r): # generate all possible combinations of items
            total_weight = sum(weights[i] for i in subset) # compute the total weight of this combination
            total_value = sum(values[i] for i in subset)  # compute the total value of this combination
            if total_weight <= capacity and total_value > best_value: # if this combination is feasible and better than the best one found so far
                best_value = total_value # update the best value
                best_combination = subset # update the best combination

      return best_value, best_combination


In [30]:
solutions = []
start_time_gen_and_test = time.time()
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    solutions.append(1 if target == solution else 0)
end_time_gen_and_test = time.time()


In [31]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))


Accuracy of best prices found is 1.0


**Your Analysis:**

In [32]:
print("Time taken: ", end_time_gen_and_test - start_time_gen_and_test)

Time taken:  0.25493407249450684


We know the complexity of brute force algorithm for this problem is O(n^2), and We can see it tooks about 0.28 second to execute. It will be impossible to finish if the dataset is much larger. However, it is the only algorithm that can reach 100% accuracy

**5. Greedy Search**

We will have Greedy Search in this part and see how it goes.

In [33]:
def greedy_by_criterion(weights, prices, capacity, criterion):
    """
    Greedy algorithm for the knapsack problem based on a given criterion.
    
    Args:
    - weights (list): List of item weights.
    - prices (list): List of item prices.
    - capacity (int): Total capacity of the knapsack.
    - criterion (list): Sorted list of items based on the chosen criterion.
    
    Returns:
    - int: Total price of the chosen items.
    - list: Binary representation of the chosen items.
    """
    current_weight = 0
    current_price = 0
    current_solution = []

    for idx in criterion:
        if current_weight + weights[idx] <= capacity:
            current_weight += weights[idx]
            current_price += prices[idx]
            current_solution.append(idx)
    
    binary_representation = [0.] * len(prices)
    for idx in current_solution:
        binary_representation[idx] = 1.

    return current_price, binary_representation


def greedy(data):
    weights = data['Weights']
    prices = data['Prices']
    capacity = data['Capacity']

    # Initialize variables for debugging (so we can comment out any logic of this greedy algorithm to see their impact to the correctness of the algorithm) 
    price_val, price_sol = 0, [0.] * len(prices)
    weight_val, weight_sol = 0, [0.] * len(prices)
    pw_ratio_val, pw_ratio_sol = 0, [0.] * len(prices)

    # Criteria for sorting
    price_sorted_indices = sorted(range(len(prices)), key=lambda k: prices[k], reverse=True) # sort by price in descending order
    weight_sorted_indices = sorted(range(len(weights)), key=lambda k: weights[k]) # sort by weight in ascending order
    pw_ratio_sorted_indices = sorted(range(len(prices)), key=lambda k: prices[k] / weights[k], reverse=True) # sort by price/weight ratio in descending order

    # Apply greedy strategy for each criterion 
    price_val, price_sol = greedy_by_criterion(weights, prices, capacity, price_sorted_indices)  # run greedy algorithm based on price
    weight_val, weight_sol = greedy_by_criterion(weights, prices, capacity, weight_sorted_indices) # run greedy algorithm based on weight
    pw_ratio_val, pw_ratio_sol = greedy_by_criterion(weights, prices, capacity, pw_ratio_sorted_indices) # run greedy algorithm based on price/weight ratio

    # Return the best solution among the three strategies
    if price_val >= weight_val and price_val >= pw_ratio_val:
        return price_val, price_sol
    elif weight_val >= price_val and weight_val >= pw_ratio_val:
        return weight_val, weight_sol
    else:
        return pw_ratio_val, pw_ratio_sol


In [34]:
solutions_greedy = []
start_time_greedy = time.time()
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)
end_time_greedy = time.time()


In [35]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

Greedy Accuracy is 0.9901811924283834


In [36]:
print("Time taken: ", end_time_greedy - start_time_greedy)

Time taken:  0.1490631103515625


**Your Analysis:**

While addressing this specific knapsack problem, I employed three distinct greedy strategies: prioritizing by highest price, by least weight, and by the price-to-weight ratio. By integrating these three strategies, we achieved an accuracy of approximately 99% with an execution time around 0.55 seconds. This is significantly faster than exhaustive search while maintaining high precision.

Diving deeper into the contribution of each strategy, we observe:

- **By Highest Price**: When employed exclusively, this emerges as the most effective strategy among the trio with an accuracy of 90.67%.
- **By Price-to-Weight Ratio**: Used in isolation, this strategy records an accuracy of 83.42%. While not as effective as the highest price strategy, it remains a relatively potent method.
- **By Least Weight**: Using only this approach yields an accuracy of 51.10%. Thus, this strategy seems to have a relatively lower contribution to the dataset, adding about 1% to the accuracy when combined with others.

In conclusion, the greedy strategy offers a method balancing time and accuracy. However, for this specific dataset, the strategy prioritizing the least weight appears to be redundant.




**6. Simulated Annealing**

We are going to implement Simulated Annealing algorithm in this part, and see its performance and behavior.


In [37]:
import random
import math

def greedy_initial_solution(data):
    """Get the initial solution using the greedy strategy based on price."""
    weights = data['Weights']
    prices = data['Prices']
    capacity = data['Capacity']
    
    # Sort items based on price, in descending order
    price_sorted_indices = sorted(range(len(prices)), key=lambda k: prices[k], reverse=True)
    
    # Initialize solutions results
    total_weight = 0
    total_price = 0
    solution = [0] * len(weights)  
    
    for idx in price_sorted_indices:
        if total_weight + weights[idx] <= capacity:
            total_weight += weights[idx]
            total_price += prices[idx]
            solution[idx] = 1
    
    return solution, total_price

def energy(solution, weights, prices, capacity):
    """Compute the energy (fitness) of a solution."""
    total_weight = sum([w * s for w, s in zip(weights, solution)])
    total_price = sum([p * s for p, s in zip(prices, solution)])

    # If solution exceeds capacity, return 0
    if total_weight > capacity:
        return 0
    return total_price

def accept(deltaE, T):
    """Acceptance function."""
    if deltaE > 0:
        return True
    else:
        r = random.random()
        return r < math.exp(deltaE / T)
    
def generate_new_solution_based_on_PWratio(solution, weights, prices, capacity):
    """
    Generate a new solution by considering price-to-weight ratio and introducing some randomness.
    """
    # Copy the current solution
    new_solution = solution.copy()
    
    # Randomly select an item from the current solution
    selected_items = [i for i, val in enumerate(solution) if val == 1]
    if not selected_items:
        return solution
    item_to_remove = random.choice(selected_items)
    
    # Randomly select an item not in the current solution
    unselected_items = [i for i, val in enumerate(solution) if val == 0]
    if not unselected_items:
        return new_solution
    item_to_add = random.choice(unselected_items)
    
    # Compute the value-to-weight ratios
    ratio_old = prices[item_to_remove] / weights[item_to_remove]
    ratio_new = prices[item_to_add] / weights[item_to_add]
    
    # Replace the item if the new item has a better ratio or based on a probability
    if ratio_new > ratio_old or random.random() < 0.2:
        new_solution[item_to_remove] = 0
        new_solution[item_to_add] = 1
    
    return new_solution

def generate_new_solution_based_on_temperature(solution, T):
    """Generate a new candidate solution by flipping multiple bits based on temperature."""
    # Max number of bits we can flip
    max_bits_to_flip = len(solution)
    
    # Dynamically determine the number of bits to flip based on temperature
    # When T is high, we can flip more bits. As T reduces, the number of bits we flip reduces.
    num_bits_to_flip = int(T * max_bits_to_flip)
    
    # If num_bits_to_flip becomes 0, set it to 1 to ensure we flip at least one bit
    num_bits_to_flip = max(1, num_bits_to_flip)
    
    x_new = solution.copy()
    
    # Randomly choose bits to flip
    indices_to_flip = random.sample(range(len(solution)), num_bits_to_flip)
    
    for idx in indices_to_flip:
        x_new[idx] = 1 - x_new[idx]
    
    return x_new




def simulated_annealing(data, N , initial_temperature, cooling_rate):
    """Simulated Annealing for the Knapsack Problem."""
    weights = data['Weights']
    prices = data['Prices']
    capacity = data['Capacity']
    Tmin = 0.05  # Define a minimum temperature for termination
    " The value minimun temperature is really important. We test it by 0.05, 0.01, 0.001 , and found that 0.05 is the best by balancing the run time and accuary."
    Eth = 0  # Energy threshold
    
    T = initial_temperature
    # Using greedy strategy to get the initial solution
    X, E = greedy_initial_solution(data)
    
    while T > Tmin and E > Eth:
        
        # Generate a new candidate solution by using price-to-weight ratio
        # it is bad, cause its limited solution space/Preferences for local optimization/lack of randomness
        # x_new = generate_new_solution_based_on_PWratio(X, weights, prices, capacity)

        # Generate a new candidate solution by flipping multiple random bits
        # it is better than the previous one, cause it has more randomness, larger search space, can jump out of local optimum at 
        # high temperature
        x_new = generate_new_solution_based_on_temperature(X, T)
        
        E_new = energy(x_new, weights, prices, capacity)
        deltaE = E_new - E
        
        # If the new solution is better or accepted by the Accept function
        if accept(deltaE, T):
            X = x_new
            E = E_new
            
        # Reduce the temperature
        T = T * cooling_rate
    
    return E, X




  


In [38]:
solutions_sa = []
solution_list = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 10, initial_temperature=1, cooling_rate=0.95)
    solutions_sa.append(1 if target == solution else 0)


In [39]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))


Simulated Annealing Accuracy is 0.9464520700475757


**Your Analysis:**

### Analysis
In the Simulated Annealing algorithm, the strategy for generating new candidate solutions and the value of the minimum temperature play critical roles in determining the efficiency and effectiveness of the search process. We compared two distinct strategies for generating new solutions in the context of the Knapsack Problem:

1. Price-to-Weight Ratio-Based Strategy: This strategy followed the idea of prioritizing items based on their price-to-weight ratio. By swapping items with differing ratios, this method aimed to improve the quality of the solution.

2. Multiple Bit-Flipping Strategy: This approach introduced more randomness into the generation of new candidates. By flipping multiple bits based on the current temperature, the algorithm could explore a wider range of solutions, especially during the early stages when the temperature was high.

### Findings:
- While the Price-to-Weight Ratio-Based Strategy is intuitive, it exhibited limited exploration capabilities. This often led the algorithm to get trapped in local optima. By predominantly focusing on the value-to-weight ratio, it potentially overlooked other promising regions of the solution space. As a result, the accuracy of the algorithm was slightly lower than that of the Greedy Search, which only used the highest price strategy, even though we employed it as the starting solution.

- In contrast, the Multiple Bit-Flipping Strategy provided broader exploration potential. Its inherent randomness allowed the algorithm to escape local optima, leading to a more diverse set of solutions. With this strategy, there was a noticeable improvement in runtime (from 14.5s to 11.7s) and a slight increase in accuracy (from 0.89 to 0.94).

- The value of the minimum temperature also played a crucial role in the algorithm's performance. If set too high, the algorithm risked getting trapped in local optima. However, if set too low, the runtime dramatically increased without a commensurate rise in accuracy. In our experiments, we found the best performance when the minimum temperature was set to 0.05, balancing runtime (around 11.7s) and accuracy (around 0.94). Reducing the minimum temperature to 0.01 led to a marginal accuracy increase of 0.005 but increased runtime to 16.7s. Notably, setting it to 0.001 almost doubled the runtime to 23.8s without a significant accuracy boost.

### Conclusion:
Both the choice of new solution generation strategy and the value of minimum temperature are crucial in Simulated Annealing. While domain-specific knowledge, like the value-to-weight ratio in the Knapsack Problem, can be beneficial, introducing elements of randomness and broader exploration, as seen with the Multiple Bit-Flipping strategy, can significantly enhance the algorithm's performance. Experimenting with and adjusting these strategies based on the specific problem and dataset is key to harnessing the full potential of Simulated Annealing.

**7. Genetic Algorithm**

We will implement Genetic Algorithm in this part, and see its performance and behavior and compare it to other algorithms.


In [42]:
import random

def calculate_fitness(ind, data):
    weights = data['Weights']
    prices = data['Prices']
    capacity = data['Capacity']
    """Calculate the fitness of an individual."""
    total_weight = sum([w * s for w, s in zip(weights, ind)])
    total_price = sum([p * s for p, s in zip(prices, ind)])
    
    # If solution exceeds capacity, return 0
    if total_weight > capacity:
        return 0
    return total_price

def select_next_generation(population, offspring, population_size,data):
    """
    Selects the next generation from the current population and offspring.

    :param population: The current population.
    :param offspring: The offspring generated from crossover and mutation.
    :param population_size: The fixed size of the population.
    :return: The next generation.
    """
    # Combine the population and offspring
    combined = population + offspring
    
    # Sort the combined list based on fitness (assuming higher is better)
    sorted_combined = sorted(combined, key=lambda individual:calculate_fitness(individual,data), reverse=True)
    
    # Select the top 'population_size' individuals
    next_generation = sorted_combined[:population_size]
    
    return next_generation

 

def crossover(parent1, parent2, cross_rate):
  # Determine the length of the parents
    length = len(parent1)
    
    # Initialize children as the parents
    child1 = parent1.copy()
    child2 = parent2.copy()
    
    # Check if crossover should occur
    if random.random() < cross_rate:
        # Select a crossover point randomly
        cross_point = random.randint(0, length - 1)
        
        # Swap genes after the crossover point
        for i in range(cross_point, length):
            child1[i], child2[i] = child2[i], child1[i]

    return child1, child2

def mutation(child, mut_rate):
  return [child[i] if random.random() > mut_rate else 1-child[i] for i in range(len(child))]

def initialize_population(population_size, num_items):
    """
    Initializes a population for the knapsack problem.
    """
    population = []
    for _ in range(population_size):
        individual = [random.choice([0, 1]) for _ in range(num_items)]  # 直接使用整数列表
        population.append(individual)
    return population

def tournament_selection(population, k, num_winners,data):
    """
    Selects a subset of the population using tournament selection.

    :param population: The entire population.
    :param k: Number of individuals to compete in each tournament.
    :param num_winners: Total number of winners (parents) to select. 
    :return: A subset of the population.
    """
    winners = []
    for _ in range(num_winners):
        competitors = random.sample(population, k)
        winner = max(competitors, key=lambda individual: calculate_fitness(individual,data))
        winners.append(winner)

    return winners


def genetic_algorithm(data, population_size=50, num_generations=50, mut_rate=0.1, cross_rate=0.7, tournament_size=5):
    initialized_population = initialize_population(population_size, 5)
    weights = data['Weights']
    prices = data['Prices']
    capacity = data['Capacity']

    best_solution_price = 0
    best_solution = []
    num_crossover = int(cross_rate * population_size) // 2
    num_parents = 2 * num_crossover

    for _ in range(num_generations): 
        child_list = []
        elite_elements = tournament_selection(initialized_population, tournament_size, num_parents,data)
        
        for _ in range(num_crossover): 
            parent1 = elite_elements.pop(random.choice(range(len(elite_elements))))
            parent2 = elite_elements.pop(random.choice(range(len(elite_elements))))
            new_child1, new_child2 = crossover(parent1, parent2, cross_rate)
            child_list.append(mutation(new_child1, mut_rate))
            child_list.append(mutation(new_child2, mut_rate))
        
        initialized_population = select_next_generation(initialized_population, child_list, population_size,data)
        
        # 检查是否找到更好的解决方案
        for individual in initialized_population:
            fitness = calculate_fitness(individual, data)
            if fitness > best_solution_price :
                best_solution_price = fitness
                best_solution = individual

    return best_solution_price, best_solution

In [45]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 25, num_generations = 25, mut_rate = 0.15, cross_rate = 0.6, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)


In [46]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

Genetic Algorithm Accuracy is 0.9888652697641461


**Your Analysis:**

------------------------------------------------------------------------------------------------

**8. Comparative Study**

description  +  tables/figures

--------------------------------------------------------------------------


**9. Conclusion**

Comment on the empirical study, its results, and give ideas for future work.

--------------------------------------------------------------------------


**10 References**

Make sure you provide references to ALL sources used (articles, code, algorithms).

**Hint:** To share a link to your colab notebook, click on "share" on the top right. Then, under *General access* , change *Restricted* to "Anyone with the link".